# Caractérisation et préprocessing des exemples NL -> Python 
### Overleaf disponible ici : https://www.overleaf.com/7663237845srdhzcgvqdwg

In [1]:
import pandas as pd
import nltk
from pathlib import Path
from dataset.data_conala import *
import numpy as np
import glob
import pickle
import ast
import astor
from tokenize import tokenize, untokenize, COMMENT, TokenError
from io import BytesIO
from asdl.ast_operation import *

## Load Grammar

In [2]:
asdl_text = open('./asdl/grammar.txt').read()
print(asdl_text)

grammar, _, _ = Grammar.from_text(asdl_text)
act_list = [GrammarRule(rule.constructor.name, rule.type.name, rule.fields) for rule in grammar]
Reduce = ReduceAction('Reduce')
act_dict = dict([(act.label, act) for act in act_list])

identifier, int, string, bytes, object, singleton, constant

    mod = Module(stmt* body)
        | Interactive(stmt* body)
        | Expression(expr body)
        | Suite(stmt* body)

    stmt = FunctionDef(identifier name, arguments args,
                       stmt* body, expr* decorator_list, expr? returns)
          | AsyncFunctionDef(identifier name, arguments args,
                             stmt* body, expr* decorator_list, expr? returns)

          | ClassDef(identifier name,
             expr* bases,
             keyword* keywords,
             stmt* body,
             expr* decorator_list)
          | Return(expr? value)

          | Delete(expr* targets)
          | Assign(expr* targets, expr value)
          | AugAssign(expr target, operator op, expr value)

          | AnnAssign(expr target, expr annotation, expr? value, int simple)

          | For(expr target, expr iter, stmt* body, stmt* orelse)
          | AsyncFor(expr target, expr iter, stmt* body, stmt* orelse)
 

# CodeSearchNet (Github)

## Load dataset

In [4]:
python_files = sorted(Path('./dataset/data_github/python/').glob('**/*.gz'))

print(len(python_files))

codesearchnet_columns=['docstring', 'docstring_tokens', 'code', 'code_tokens', 'partition']

def jsonl_list_to_dataframe(file_list, columns=codesearchnet_columns):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f, 
                                   orient='records', 
                                   compression='gzip',
                                   lines=True)[columns] 
                      for f in file_list], sort=False)

16


In [5]:
pydf = jsonl_list_to_dataframe(python_files)

In [15]:
pydf_doc = pydf['docstring']
from docstring_parser import parse

def create_dict(string):
    replacement = {}
    doc_str = parse(x)
    params = [param.arg_name for param in doc_str.params]
    for param in params:
        replacement[param] = '`'+param+'`'
    return replacement

def multiple_replace(doc_str):
    replacement = {}
    print(doc_str)
    doc_str_parsed = parse(doc_str)
    params = [param.arg_name for param in doc_str_parsed.params]
    for param in params:
        replacement[param] = '`'+param+'`'
    if replacement == {}:
        return doc_str
    else:
        print(replacement)
        # Create a regular expression  from the dictionary keys
        regex = re.compile("(%s)" % "|".join(map(re.escape, replacement.keys())))

        # For each match, look-up corresponding value in dictionary
        return regex.sub(lambda mo: replacement[mo.string[mo.start():mo.end()]], doc_str) 

for x in pydf_doc.iloc[:5000]:
    print(x)
    doc_str = multiple_replace(x)
    print(doc_str)


Extracts video ID from URL.
Extracts video ID from URL.
Extracts video ID from URL.
str->list
    Convert XML to URL List.
    From Biligrab.
str->list
    Convert XML to URL List.
    From Biligrab.
str->list
    Convert XML to URL List.
    From Biligrab.
From http://cdn37.atwikiimg.com/sitescript/pub/dksitescript/FC2.site.js
    Also com.hps.util.fc2.FC2EncrptUtil.makeMimiLocal
    L110
From http://cdn37.atwikiimg.com/sitescript/pub/dksitescript/FC2.site.js
    Also com.hps.util.fc2.FC2EncrptUtil.makeMimiLocal
    L110
From http://cdn37.atwikiimg.com/sitescript/pub/dksitescript/FC2.site.js
    Also com.hps.util.fc2.FC2EncrptUtil.makeMimiLocal
    L110
wrapper
wrapper
wrapper
Downloads Dailymotion videos by URL.
Downloads Dailymotion videos by URL.
Downloads Dailymotion videos by URL.
http://stackoverflow.com/a/30923963/2946714
http://stackoverflow.com/a/30923963/2946714
http://stackoverflow.com/a/30923963/2946714
video page
video page
video page
course page
course page
course page
D

ValueError: not enough values to unpack (expected 2, got 1)

In [78]:
pydf.head(3)

,docstring,docstring_tokens,code,code_tokens,partition
0,Extracts video ID from URL.,"[Extracts, video, ID, from, URL, .]","def get_vid_from_url(url):\n """"""Extract...","[def, get_vid_from_url, (, url, ), :, return, ...",test
1,str->list\n Convert XML to URL List.\n F...,"[str, -, >, list, Convert, XML, to, URL, List,...","def sina_xml_to_url_list(xml_data):\n """"""st...","[def, sina_xml_to_url_list, (, xml_data, ), :,...",test
2,From http://cdn37.atwikiimg.com/sitescript/pub...,"[From, http, :, //, cdn37, ., atwikiimg, ., co...","def makeMimi(upid):\n """"""From http://cdn37....","[def, makeMimi, (, upid, ), :, strSeed, =, ""gG...",test


In [23]:
index_example = 10

code_example = pydf.iloc[index_example]['code']
print(code_example)

def sina_download(url, output_dir='.', merge=True, info_only=False, **kwargs):
    """Downloads Sina videos by URL.
    """
    if 'news.sina.com.cn/zxt' in url:
        sina_zxt(url, output_dir=output_dir, merge=merge, info_only=info_only, **kwargs)
        return

    vid = match1(url, r'vid=(\d+)')
    if vid is None:
        video_page = get_content(url)
        vid = hd_vid = match1(video_page, r'hd_vid\s*:\s*\'([^\']+)\'')
        if hd_vid == '0':
            vids = match1(video_page, r'[^\w]vid\s*:\s*\'([^\']+)\'').split('|')
            vid = vids[-1]

    if vid is None:
        vid = match1(video_page, r'vid:"?(\d+)"?')
    if vid:
        #title = match1(video_page, r'title\s*:\s*\'([^\']+)\'')
        sina_download_by_vid(vid, output_dir=output_dir, merge=merge, info_only=info_only)
    else:
        vkey = match1(video_page, r'vkey\s*:\s*"([^"]+)"')
        if vkey is None:
            vid = match1(url, r'#(\d+)')
            sina_download_by_vid(vid, output_dir=output_di

In [143]:
index = 5000

print(pydf.iloc[index]['code'])
print()
print()
print(pydf.iloc[index]['docstring'])

def metadata(self, item, filter_classified=False):
        """Add metadata to an item.

        It adds metadata to a given item such as how and
        when it was fetched. The contents from the original item will
        be stored under the 'data' keyword.

        :param item: an item fetched by a backend
        :param filter_classified: sets if classified fields were filtered
        """
        item = {
            'backend_name': self.__class__.__name__,
            'backend_version': self.version,
            'perceval_version': __version__,
            'timestamp': datetime_utcnow().timestamp(),
            'origin': self.origin,
            'uuid': uuid(self.origin, self.metadata_id(item)),
            'updated_on': self.metadata_updated_on(item),
            'classified_fields_filtered': self.classified_fields if filter_classified else None,
            'category': self.metadata_category(item),
            'tag': self.tag,
            'data': item,
        }

        return 

## Statistics

### Number of examples

In [7]:
pydf.partition.value_counts()

train    412178
valid     23107
test      22176
Name: partition, dtype: int64

### Token lengths

In [88]:
pydf['code_len'] = pydf.code_tokens.apply(lambda x: len(x))
pydf['query_len'] = pydf.docstring_tokens.apply(lambda x: len(x))

In [90]:
print('Avg docstring len: %d' % np.average(pydf['query_len']), file=sys.stderr)
print('Avg code len: %d' % np.average(pydf['code_len']), file=sys.stderr)

Avg docstring len: 16
Avg code len: 117


In [9]:
query_len_summary = pydf['query_len'].quantile([.5, .7, .8, .9, .95])
display(pd.DataFrame(query_len_summary))

,query_len
0.50,10.0
0.70,15.0
0.80,20.0
0.90,33.0
0.95,48.0


In [10]:
code_len_summary = pydf['code_len'].quantile([.5, .7, .8, .9, .95])
display(pd.DataFrame(code_len_summary))

,code_len
0.50,72.0
0.70,114.0
0.80,155.0
0.90,237.0
0.95,341.0


#### Get action sequence & reconstruct code 

In [11]:
py_ast = ast.parse(code_example)

action_sequence = [(a, flag) for a, flag in  ast2seq(py_ast, act_dict, [], [], [])[0]]

print('action_sequence len of the example', len(action_sequence))

ast_reconstructed = seq2ast(make_iterlists(deque(action_sequence)))

print('\n Reconstructed code : \n \n', astor.to_source(ast_reconstructed).rstrip())

action_sequence len of the example 369

 Reconstructed code : 
 
 def sina_download(url, output_dir='.', merge=True, info_only=False, **kwargs):
    """Downloads Sina videos by URL.
    """
    if 'news.sina.com.cn/zxt' in url:
        sina_zxt(url, output_dir=output_dir, merge=merge, info_only=
            info_only, **kwargs)
        return
    vid = match1(url, 'vid=(\\d+)')
    if vid is None:
        video_page = get_content(url)
        vid = hd_vid = match1(video_page, "hd_vid\\s*:\\s*\\'([^\\']+)\\'")
        if hd_vid == '0':
            vids = match1(video_page, "[^\\w]vid\\s*:\\s*\\'([^\\']+)\\'"
                ).split('|')
            vid = vids[-1]
    if vid is None:
        vid = match1(video_page, 'vid:"?(\\d+)"?')
    if vid:
        sina_download_by_vid(vid, output_dir=output_dir, merge=merge,
            info_only=info_only)
    else:
        vkey = match1(video_page, 'vkey\\s*:\\s*"([^"]+)"')
        if vkey is None:
            vid = match1(url, '#(\\d+)')
       

#### Stats on action 

In [75]:
def get_ast(code):
    try: 
        return(ast.parse(code))
    except:
        pass

pydf['ast'] = pydf.code.apply(get_ast)
pydf['action'] = pydf.ast.apply(ast2seq, action_dict=act_dict, parent_type=[], parent_field=[], parent_cardinality=[])

In [76]:
action_lens = [len(e[0]) for e in pydf['action']]

In [77]:
print('Max action len: %d' % max(action_lens), file=sys.stderr)
print('Avg action len: %d' % np.average(action_lens), file=sys.stderr)
print('Actions larger than 100: %d' % len(list(filter(lambda x: x > 100, action_lens))), file=sys.stderr)
print('Actions larger than 200: %d' % len(list(filter(lambda x: x > 200, action_lens))), file=sys.stderr)
print('Actions larger than 300: %d' % len(list(filter(lambda x: x > 300, action_lens))), file=sys.stderr)
print('Actions larger than 1000: %d' % len(list(filter(lambda x: x > 1000, action_lens))), file=sys.stderr)

Max action len: 51906
Avg action len: 219
Actions larger than 100: 302963
Actions larger than 200: 150254
Actions larger than 300: 85524
Actions larger than 1000: 8429


### Docstring from the dataset does not contain comments inside the function

In [14]:
print(pydf.iloc[index_example]['docstring'])

Downloads Sina videos by URL.


### Get comments inside the function

In [83]:
comments = []

code_encoded = code_example.encode('utf-8')
code_bytes = BytesIO(code_encoded) # Bytes form to use tokenize 
code_tokenized = tokenize(code_bytes.readline)

for type_id, string_value, _, _, _ in code_tokenized:
    if type_id == 55:
        comments.append(string_value)
        
print(comments)

["#title = match1(video_page, r'title\\s*:\\s*\\'([^\\']+)\\'')"]


### Delete docstrings from source code

In [86]:
parsed = ast.parse(code_example)

for node in ast.walk(parsed):
    # let's work only on functions & classes definitions
    if not isinstance(node, (ast.FunctionDef, ast.ClassDef, ast.AsyncFunctionDef)):
        continue

    if not len(node.body):
        continue

    if not isinstance(node.body[0], ast.Expr):
        continue

    if not hasattr(node.body[0], 'value') or not isinstance(node.body[0].value, ast.Str):
        continue

    # Uncomment lines below if you want print what and where we are removing
    # print(node)
    # print(node.body[0].value.s)

    node.body = node.body[1:]
    
print('***** Processed source code output ******\n=========================================')

print(astor.to_source(parsed))

***** Processed source code output ******
def sina_download(url, output_dir='.', merge=True, info_only=False, **kwargs):
    if 'news.sina.com.cn/zxt' in url:
        sina_zxt(url, output_dir=output_dir, merge=merge, info_only=
            info_only, **kwargs)
        return
    vid = match1(url, 'vid=(\\d+)')
    if vid is None:
        video_page = get_content(url)
        vid = hd_vid = match1(video_page, "hd_vid\\s*:\\s*\\'([^\\']+)\\'")
        if hd_vid == '0':
            vids = match1(video_page, "[^\\w]vid\\s*:\\s*\\'([^\\']+)\\'"
                ).split('|')
            vid = vids[-1]
    if vid is None:
        vid = match1(video_page, 'vid:"?(\\d+)"?')
    if vid:
        sina_download_by_vid(vid, output_dir=output_dir, merge=merge,
            info_only=info_only)
    else:
        vkey = match1(video_page, 'vkey\\s*:\\s*"([^"]+)"')
        if vkey is None:
            vid = match1(url, '#(\\d+)')
            sina_download_by_vid(vid, output_dir=output_dir, merge=merge,
  

### Number of examples with comments

In [72]:
def is_commented(code):
    try:
        code_encoded = code.encode('utf-8')
        code_bytes = BytesIO(code_encoded)
        code_tokenized = tokenize(code_bytes.readline)
        for type_id, string_value, _, _, _ in code_tokenized:
            if type_id == 55:
                return True
        else:
            return False
    except:
        return False

pydf['is_commented'] = pydf.code.apply(is_commented)
pydf.is_commented.value_counts()

False    308230
True     149231
Name: is_commented, dtype: int64

# CoNaLa (Stackoverflow)

## Load dataset

In [5]:
conala_columns = ['intent', 'snippet_actions', 'snippet_tokens']

file_list = ['./dataset/data_conala/conala-corpus/conala-train.json.seq2seq', './dataset/data_conala/conala-corpus/conala-test.json.seq2seq']

file_list_mined = ['./dataset/data_conala/conala-corpus/conala-train.json.seq2seq', './dataset/data_conala/train/conala-val.csv', './dataset/data_conala/conala-corpus/conala-mined.json.seq2seq']

data_train = pickle.load(open('./dataset/data_conala/conala-corpus/conala-train.json.seq2seq', 'rb'))

data_test = pickle.load(open('./dataset/data_conala/conala-corpus/conala-test.json.seq2seq', 'rb'))

data_mined = pickle.load(open('./dataset/data_conala/conala-corpus/conala-mined.jsonl.seq2seq', 'rb'))

In [5]:
# tranx_action = [example['snippet_actions'] for example in data_train] + [example['snippet_actions'] for example in data_test]
tranx_action = [example['snippet_actions'] for example in data_train]

all_action = tranx_action + [example['snippet_actions'] for example in data_mined]

In [6]:
# tranx_code = [example['snippet_tokens'] for example in data_train] + [example['snippet_tokens'] for example in data_test]
tranx_code = [example['snippet_tokens'] for example in data_train]

all_code = tranx_action + [example['snippet_tokens'] for example in data_mined]

In [8]:
#tranx_query = [example['rewritten_intent'] for example in data_train] + [example['rewritten_intent'] for example in data_test]
tranx_query = [example['rewritten_intent'] for example in data_train]
tranx_query = [nltk.word_tokenize(example) for example in tranx_query if isinstance(example, str)]

all_queries = tranx_action + [example['intent'] for example in data_mined]
all_queries = [nltk.word_tokenize(example) for example in all_queries if isinstance(example, str)]


## Statistics

### Stats on dataset

In [9]:
tranx_query_len = [len(query) for query in tranx_query]
all_query_len = [len(query) for query in all_queries]

print('Max clean query_len example: %d' % max(tranx_query_len), file=sys.stderr)
print('Avg clean query_len example: %d' % np.average(tranx_query_len), file=sys.stderr)
print('Var clean query_len example: %d' % np.var(tranx_query_len), file=sys.stderr)

print('Max all examples query_len: %d' % max(all_query_len), file=sys.stderr)
print('Avg all examples query_len: %d' % np.average(all_query_len), file=sys.stderr)


Max clean query_len example: 56
Avg clean query_len example: 14
Var clean query_len example: 44
Max all examples query_len: 34
Avg all examples query_len: 9


In [10]:
tranx_code_len = [len(code) for code in tranx_code]
all_code_len = [len(code) for code in all_code]

print('Max clean code_len example: %d' % max(tranx_code_len), file=sys.stderr)
print('Avg clean code_len example: %d' % np.average(tranx_code_len), file=sys.stderr)
print('Var clean code_len example: %d' % np.var(tranx_code_len), file=sys.stderr)

print('Max all examples code_len: %d' % max(all_code_len), file=sys.stderr)
print('Avg all examples code_len: %d' % np.average(all_code_len), file=sys.stderr)

Max clean code_len example: 61
Avg clean code_len example: 17
Var clean code_len example: 77
Max all examples code_len: 6527
Avg all examples code_len: 23


In [11]:
tranx_action_len = [len(action) for action in tranx_action]

all_action_len = [len(action) for action in all_action]

print('Number of clean examples actions: %d' % len(tranx_action_len), file=sys.stderr)
print('Max clean action_len example: %d' % max(tranx_action_len), file=sys.stderr)
print('Avg clean action_len example: %d' % np.average(tranx_action_len), file=sys.stderr)
print('Var clean action_len example: %d' % np.var(tranx_action_len), file=sys.stderr)
print('Actions larger than 100 clean example: %d' % len(list(filter(lambda x: x > 100, tranx_action_len))), file=sys.stderr)
print('Number of raw examples: %d' % len(all_action_len), file=sys.stderr)
print('Max all examples action_len: %d' % max(all_action_len), file=sys.stderr)
print('Avg all examples action_len: %d' % np.average(all_action_len), file=sys.stderr)
print('Action sequence larger than 100 within all examples: %d' % len(list(filter(lambda x: x > 100, all_action_len))), file=sys.stderr)
print('Action sequence larger than 200 within all examples: %d' % len(list(filter(lambda x: x > 200, all_action_len))), file=sys.stderr)
print('Action sequence larger than 300 within all examples: %d' % len(list(filter(lambda x: x > 300, all_action_len))), file=sys.stderr)

Number of clean examples actions: 2371
Max clean action_len example: 112
Avg clean action_len example: 24
Var clean action_len example: 152
Actions larger than 100 clean example: 1
Number of raw examples: 591167
Max all examples action_len: 11238
Avg all examples action_len: 36
Action sequence larger than 100 within all examples: 33507
Action sequence larger than 200 within all examples: 6683
Action sequence larger than 300 within all examples: 1540


### Exemple de mapping de variables

#### Différence entre les exemples initiaux et prétraités à la main

In [6]:
df_train = pd.read_json('./dataset/data_conala/conala-corpus/conala-train.json')

initial_intent = df_train.iloc[37]['intent']
rewritten_intent = df_train.iloc[37]['rewritten_intent']
code = df_train.iloc[37]['snippet']

print('Variable names are between quotes ` `, for example:')
print(rewritten_intent)
print()
print('The initial intent was:')
print(initial_intent)
print()
print('The initial code is:')
print(code)

Variable names are between quotes ` `, for example:
append the sum of each tuple pair in the grouped list `list1` and list `list2` elements to list `list3`

The initial intent was:
How do i add two lists' elements into one list?

The initial code is:
list3 = [(a + b) for a, b in zip(list1, list2)]


#### Mapping d'exemple dans le cas où il est "cleené"

In [68]:
canonical_intent, slot_map = canonicalize_intent(rewritten_intent)
lower_intent = canonical_intent.lower()
print('The intent becomes:')
print(lower_intent)
intent_tokens = nltk.word_tokenize(lower_intent)

The intent becomes:
append the sum of each tuple pair in the grouped list var_0 and list var_1 elements to list var_2


In [69]:
encoded_reconstr_code_canonical = canonicalize_code(code, slot_map)

print('Then the code switches to:')
print(encoded_reconstr_code_canonical)

Then the code change to:
var_2 = [(a + b) for a, b in zip(var_0, var_1)]


In [147]:
canonical_intent, slot_map = canonicalize_intent("create list `done` containing permutations of each element in list `[a, b, c, d]` with variable `x` as tuples")
lower_intent = canonical_intent.lower()
print('The intent becomes:')
print(lower_intent)
print(slot_map)

The intent becomes:
create list var_0 containing permutations of each element in list str_0 with variable var_1 as tuples
{'var_0': {'value': 'done', 'quote': '`', 'type': 'var'}, 'str_0': {'value': '[a, b, c, d]', 'quote': '`', 'type': 'str'}, 'var_1': {'value': 'x', 'quote': '`', 'type': 'var'}}


In [146]:
encoded_reconstr_code_canonical = canonicalize_code("done = [(el, x) for el in [a, b, c, d]]", slot_map)

print('Then the code switches to:')
print(encoded_reconstr_code_canonical)

Then the code switches to:
var_0 = [(el, var_1) for el in [str_0]]


In [39]:
train_set_conala = pd.read_csv('./dataset/data_conala/train/conala-train.csv')
dev_set_conala = pd.read_csv('./dataset/data_conala/train/conala-val.csv')
test_set_django = pd.read_csv('./dataset/data_conala/test/conala-test.csv')

total_set = pd.concat([train_set_conala, dev_set_conala, test_set_django])

/Users/n.beau/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [40]:
total_variables_changed = [len(eval(example[5])) for example in total_set.values]
total_variables_changed = sum([len(eval(example)) for example in total_set['slot_map']])
print("total de variables remplacées", total_variables_changed)

total de variables remplacées 0


In [65]:

number_var_copiable = 0

for x in total_set['intent']:
    # print(x)
    quote_position = [i for i,x in enumerate(eval(x)) if x=='`']
    # print(quote_position)
    list2 = [quote_position[i:i + 2] for i in range(0, len(quote_position), 2)]
    # print(list2)
    for y in list2:
        try:
            diff = y[1] - y[0]
        except:
            diff = 0
        if diff == 2:
            number_var_copiable +=1
        else: 
            # print(x)
            pass
print(number_var_copiable)

1512


# Django

In [8]:
annot_file = './dataset/data_django/all.anno'
code_file = './dataset/data_django/all.code'

train_set_django = pd.read_csv('./dataset/data_django/train.csv')
dev_set_django = pd.read_csv('./dataset/data_django/dev.csv')
test_set_django = pd.read_csv('./dataset/data_django/test.csv')

#pydf_django_preprocess = pd.concat([train_set_django, dev_set_django, test_set_django])
pydf_django_preprocess = pd.concat([train_set_django, dev_set_django])

In [9]:
print(pydf_django_preprocess.head())

                                              intent  \
0  ['[CLS]', 'from', 'thread', '##ing', 'import',...   
1  ['[CLS]', 'import', 'module', 'warnings', '.',...   
2  ['[CLS]', 'from', 'dj', '##ango', '.', 'con', ...   
3  ['[CLS]', 'from', 'dj', '##ango', '.', 'core',...   
4  ['[CLS]', 'from', 'dj', '##ango', '.', 'core',...   

                                      snippet_tokens  \
0           ['from', 'threading', 'import', 'local']   
1                             ['import', 'warnings']   
2  ['from', 'django', '.', 'conf', 'import', 'set...   
3  ['from', 'django', '.', 'core', 'import', 'sig...   
4  ['from', 'django', '.', 'core', '.', 'cache', ...   

                                     snippet_actions slot_map  
0  ['ImportFrom', 'threading', 'alias', 'local', ...       {}  
1  ['Import', 'alias', 'warnings', 'Reduce_primit...       {}  
2  ['ImportFrom', 'django.conf', 'alias', 'settin...       {}  
3  ['ImportFrom', 'django.core', 'alias', 'signal...       {}  
4  ['I

In [10]:
tranx_query_len = [len(eval(example[0])) for example in pydf_django_preprocess.values]

print('Max clean query_len example django: %d' % max(tranx_query_len), file=sys.stderr)
print('Avg clean query_len example django: %d' % np.average(tranx_query_len), file=sys.stderr)
print('Var clean query_len example django: %d' % np.var(tranx_query_len), file=sys.stderr)

Max clean query_len example django: 143
Avg clean query_len example django: 19
Var clean query_len example django: 173


In [11]:
max_len = [eval(example[0]) for example in pydf_django_preprocess.values if len(eval(example[0])) == 135]
print(max_len)

[]


In [16]:
tranx_code_len = [len(eval(example[1])) for example in pydf_django_preprocess.values]

print('Max clean code_len example: %d' % max(tranx_code_len), file=sys.stderr)
print('Avg clean code_len example: %d' % np.average(tranx_code_len), file=sys.stderr)
print('Var clean code_len example: %d' % np.var(tranx_code_len), file=sys.stderr)

Max clean code_len example: 149
Avg clean code_len example: 10
Var clean code_len example: 81


In [17]:
tranx_action_len = [len(eval(example[2])) for example in pydf_django_preprocess.values]

print('Max clean action_len example: %d' % max(tranx_action_len), file=sys.stderr)
print('Avg clean action_len example: %d' % np.average(tranx_action_len), file=sys.stderr)
print('Var clean action_len example: %d' % np.var(tranx_action_len), file=sys.stderr)

Max clean action_len example: 148
Avg clean action_len example: 15
Var clean action_len example: 96


In [12]:
total_set_django = pd.concat([train_set_django, dev_set_django, test_set_django])
total_variables_changed = sum([len(eval(example)) for example in total_set_django['slot_map']])
print("total de variables remplacées", total_variables_changed)

total de variables remplacées 0


In [14]:
number_var_copiable = 0

for x in total_set_django['intent']:
    # print(x)
    quote_position = [i for i,x in enumerate(eval(x)) if x=='`']
    # print(quote_position)
    list2 = [quote_position[i:i + 2] for i in range(0, len(quote_position), 2)]
    # print(list2)
    for y in list2:
        try:
            diff = y[1] - y[0]
        except:
            diff = 0
        if diff == 2:
            number_var_copiable +=1
        else: 
            # print(x)
            pass
print(number_var_copiable)

5
